# Running Genesis with lume-genesis

#### In this session, we will use the lume-genesis Python package to do the following:
- load input files
- change input files
- run a genesis simulation
- load output results
- plot output results

### Some comments and parameter descriptions were taken from the [Genesis manual](https://github.com/svenreiche/Genesis-1.3-Version4/blob/master/manual). Please browse the manual as you go through the exercises.


### ***Special thanks to Irene Wang and Aditya Thapa for the LCLS-II input files!***
----------

In [ ]:
# from lume-genesis
from genesis.version4 import Genesis4
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

----

# 1. Make a run directory

In [ ]:
# Lets define our input files
lattice    = 'examples/lcls-II.lat'
input_file = 'examples/lcls-II.in'

# 2. Initialize lume-genesis

In [ ]:
# Initialize the run, make genesis object with the LCLSII template input file.
gen = Genesis4(input_file)

The ```Genesis4()``` call above loaded a premade genesis input and lattice file.   
After running this command, we can now look at what is in the input file and make changes in Python.   
All values in ```gen.input``` are parameters in the input file. 
Take a look at the information from the input file in the following cells:

# 3. Look at loaded input with Python

In [ ]:
# All inputs in Python are fields from the genesis input file.
# If there is no value in the input file template, default values are filled in.
gen.input

In [ ]:
gen.initial_particles # empty because no beam is loaded.
# You can supply genesis an initial beam distribution.

In [ ]:
dir(gen.input) # list of possible inputs you can change.

In [ ]:
gen.output # Output is empty because we have not run a simulation yet.

Next, look at the lattice info.   
There are two items in the lattice information: 
- elements = beam line components.
- filename = file with lattice definition. 

In [ ]:
# list of elements and parameters
# L = length of elements
gen.input.lattice

Genesis lattice files consist of components in the beamline (from [manual](https://github.com/svenreiche/Genesis-1.3-Version4/blob/master/manual/LATTICE.md)):

- UND - Main magnetic field (undulator)
- Dx  - Drift section
- QF  - Focusing quadrupole 
- QD  - Defocusing quadrupole 

Information included in the element descriptions: 
- L    = length of the structure in measure of the unit length.
- aw   = the dimensionless rms undulator parameter.
- nwig = number of periods. 
- lambdau = unulator period length in m.

See the manual page above for more details.

# 4. Run LCLS-II Genesis file

Genesis can output many types of data. There are several flags for data output that can be changed in the input file or Python. In these examples, we are only running steady state (time independant) simulations and using default outputs. For more details on steady state simulation, check out this example in the manual: https://github.com/svenreiche/Genesis-1.3-Version4/tree/master/examples/Example1-SteadyState.

In [ ]:
gen.verbose = True

In [ ]:
output = gen.run()

In [ ]:
# Configure the runs with changes made above
#dir(gen.configure)

# 5. Output

There are many types of output that can be saved. We leave the defaults in this case.

In [ ]:
gen.output # gives info about run time and outputs.

The run above is by default single core.   
You can run also run using mpi on multiple cores, we leave that to interested parties. 

# 6. Archiving the data

Saving the data to an h5 file allows you to reload with lume-genesis, and save most data in one file.   
You can reload your data after closing jupyterlab, and don't need to leave the browser window open.

Some notes on output files:  
fld - field history file (can get large)  
par - can get very large if many slices  
dlf (wavefront), dpa (phase space) are final field and particle files

More examples can be found here: https://github.com/slaclab/lume-genesis/tree/master/docs/examples/genesis4.

In [ ]:
# Archive data to h5 so that you can reload it later.
gen.archive('examples/lcls-II_sase.h5')

# 7. Loading data from archive file 

In [ ]:
# Output data can be loaded in a new or saved in gen object
#gen.load_archive('examples/LCLSII_sase.h5')

----

# 8. Plotting results

LUME-Genesis offers a plotting helpers on the Genesis4 object (and `Genesis4Output` itself) to work with the output data. You can specify individual data keys to plot the output data and the layout below.

### Lattice plot

View the lattice data (as interpreted by Genesis4) by interacting with the `output.lattice` object:

In [ ]:
output.lattice.plot();

In [ ]:
#Plotting beam size vs. undulator
gen.plot(["beam_xsize", "beam_ysize"])

In [ ]:
# Plot SASE power
power = gen.plot(["field_power"])

In [ ]:
# Plot log of power.
power = gen.plot(["field_power"], yscale='log')

In [ ]:
gen.plot("field_energy", y2=["field_xsize", "field_ysize"], ylim2=[0, 100e-6])

In [ ]:
len(output.load_particles())

In [ ]:
output.load_particles()

In [ ]:
output.load_particles()
pbeg = output.particles['beginning']
pend = output.particles['end'] 

In [ ]:
pbeg.plot("z", "energy", bins=200)

In [ ]:
pend.plot("z", "energy", bins=200)

In [ ]:
# Beam movie here
# change number of beam steps saved
# re-run simulation
# plot in movie

In [ ]:
for n in steps[2:]:
    #print(n)
    hold = output.particles[n]
    #data, xedge, yedge, mesh = plt.hist2d(hold.z, hold.energy, bins=200)
    hold.plot("z", "energy", bins=200)
    plt.show()
    #print(len(all_data))
    #hold.plot('z', 'energy', bins=200)
    #plt.savefig('hold'+str(n)+'.pdf')

In [ ]:
fig, ax = plt.subplots(figsize = (8,8))

#Create 2d Histogram
data,x,y = np.histogram2d(pbeg.z,pbeg.energy, bins = 200)

#Smooth with filter
im = plt.imshow(data.T)#, interpolation = 'gaussian', origin = 'lower')

#Define animation. 
# def animate(i) :
#     X = np.random.randn(100000)
#     Y = np.random.randn(100000) + 5
#     data,x,y = np.histogram2d(X,Y, bins = 15)
#     im.set_data(data)

def animate(i):
    #print(i)
    hold = output.particles[int(i)]
    data, xedge, yedge = np.histogram2d(hold.z, hold.energy, bins=200)
    im.set_data(data.T)


ani = animation.FuncAnimation(fig, animate, np.arange(0,1800,100),
                          interval = 500, blit = False)

ani.save('movie.mp4')

----